# Metrics-Based Impact Approximation

This notebook demonstrates **metrics-based impact approximation** using `evaluate_impact()`.

## Workflow Overview

1. User provides `products.csv`
2. User configures `DATA.ENRICHMENT` section
3. User calls `evaluate_impact(config.yaml)`
4. Engine handles everything internally (adapter, enrichment, transform, model)

## Setup

In [ ]:
import json
import os
import tempfile
from pathlib import Path

import yaml
from impact_engine import evaluate_impact
from online_retail_simulator import simulate

## Step 1: Create Products Catalog

In production, this would be your actual product catalog.

In [ ]:
# Create output directory
output_path = Path("output/demo_metrics_approximation")
output_path.mkdir(parents=True, exist_ok=True)

# Catalog simulation config
sim_config = {
    "STORAGE": {"PATH": str(output_path / "simulation")},
    "RULE": {
        "PRODUCTS": {
            "FUNCTION": "simulate_products_rule_based",
            "PARAMS": {"num_products": 5},
        }
    },
}

with tempfile.NamedTemporaryFile(mode="w", suffix=".yaml", delete=False) as f:
    yaml.dump(sim_config, f)
    sim_config_path = f.name

job_info = simulate(sim_config_path)
products = job_info.load_df("products")
os.unlink(sim_config_path)

products_path = output_path / "products.csv"
products.to_csv(products_path, index=False)

print(f"Created: {products_path}")
print(f"Products: {len(products)}")
products

## Step 2: Configure Metrics Approximation

Configure the impact engine with:
- **ENRICHMENT**: Quality boost parameters
- **TRANSFORM**: Prepare data for approximation
- **MODEL**: `metrics_approximation` with response function

In [3]:
# Metrics approximation configuration
config = {
    "DATA": {
        "SOURCE": {
            "type": "simulator",
            "CONFIG": {
                "mode": "rule",
                "seed": 42,
                "start_date": "2024-01-01",
                "end_date": "2024-01-14",
                "path": str(products_path),
            },
        },
        "ENRICHMENT": {
            "FUNCTION": "product_detail_boost",
            "PARAMS": {
                "enrichment_fraction": 1.0,
                "enrichment_start": "2024-01-08",
                "quality_boost": 0.15,
                "seed": 42,
            },
        },
        "TRANSFORM": {"FUNCTION": "prepare_simulator_for_approximation", "PARAMS": {}},
    },
    "MEASUREMENT": {
        "MODEL": "metrics_approximation",
        "PARAMS": {"RESPONSE": {"FUNCTION": "linear", "PARAMS": {"coefficient": 0.5}}},
    },
}

config_path = output_path / "config.yaml"
with open(config_path, "w") as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"Configuration saved to: {config_path}")
print(f"\nENRICHMENT: {config['DATA']['ENRICHMENT']['FUNCTION']}")
print(f"TRANSFORM: {config['DATA']['TRANSFORM']['FUNCTION']}")
print(f"MODEL: {config['MEASUREMENT']['MODEL']}")

Configuration saved to: output/demo_metrics_approximation/config.yaml

ENRICHMENT: product_detail_boost
TRANSFORM: prepare_simulator_for_approximation
MODEL: metrics_approximation


## Step 3: Run Impact Evaluation

A single call to `evaluate_impact()` handles everything:
- Engine creates CatalogSimulatorAdapter
- Adapter simulates metrics
- Adapter generates product_details
- Adapter applies enrichment (quality boost)
- Transform extracts quality_before/quality_after
- MetricsApproximationAdapter computes impact

In [4]:
results_path = evaluate_impact(str(config_path), str(output_path / "results"))
print(f"Results saved to: {results_path}")

Results saved to: /home/peisenha/office/business/eisenhauer-io/tools/impact-engine/documentation/notebooks/output/demo_metrics_approximation/results/job-impact-engine-20260124-065000-a59bdc83/results/impact_results.json


## Step 4: Review Results

In [5]:
with open(results_path) as f:
    results = json.load(f)

print("=" * 60)
print("METRICS-BASED IMPACT APPROXIMATION RESULTS")
print("=" * 60)

print(f"\nModel Type: {results['model_type']}")
print(f"Response Function: {results['response_function']}")

estimates = results["impact_estimates"]
print("\n--- Aggregate Impact Estimates ---")
print(f"Total Approximated Impact: ${estimates['total_approximated_impact']:.2f}")
print(f"Mean Approximated Impact:  ${estimates['mean_approximated_impact']:.2f}")
print(f"Mean Quality Change:       {estimates['mean_metric_change']:.4f}")
print(f"Number of Products:        {estimates['n_products']}")

METRICS-BASED IMPACT APPROXIMATION RESULTS

Model Type: metrics_approximation
Response Function: linear

--- Aggregate Impact Estimates ---
Total Approximated Impact: $63.85
Mean Approximated Impact:  $12.77
Mean Quality Change:       0.1546
Number of Products:        5


In [6]:
print("\n--- Per-Product Breakdown ---")
print("-" * 60)
print(f"{'Product':<20} {'Delta Quality':<15} {'Baseline':<12} {'Impact':<12}")
print("-" * 60)
for p in results["per_product"]:
    print(
        f"{p['product_id']:<20} {p['delta_metric']:<15.4f} "
        f"${p['baseline_outcome']:<11.2f} ${p['approximated_impact']:<11.2f}"
    )

print("\n" + "=" * 60)
print("Demo Complete!")
print("=" * 60)


--- Per-Product Breakdown ---
------------------------------------------------------------
Product              Delta Quality   Baseline     Impact      
------------------------------------------------------------
B1J5G5DSZP           0.1060          $215.25      $11.41      
B48ZVWATLV           0.1880          $150.00      $14.10      
BPPZF9Q9BG           0.1380          $0.00        $0.00       
BW1WIE0X55           0.2090          $314.60      $32.88      
BYZ2VSIV1G           0.1320          $82.88       $5.47       

Demo Complete!
